In [1]:
import pandas as pd
import sqlalchemy as sa  # Corrigido
from sqlalchemy import create_engine  # Importação correta para criar o engine
import os
import sys
sys.path.append(r'c:\Projetos\Python\JornadaDeDados')

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("microdados_enem_2023/DADOS/DicionarioEnem2023.csv", delimiter=";", encoding="UTF-8")

In [5]:
df.head()

,ID_QUESTIONARIO,PERGUNTA,NR_REPOSTA,DESC_RESPOSTA
0,Q001,"Até que série seu pai, ou o homem responsável ...",A,Nunca estudou.
1,Q001,"Até que série seu pai, ou o homem responsável ...",B,Não completou a 4ª série/5º ano do Ensino Fund...
2,Q001,"Até que série seu pai, ou o homem responsável ...",C,"Completou a 4ª série/5º ano, mas não completou..."
3,Q001,"Até que série seu pai, ou o homem responsável ...",D,Completou a 8ª série/9º ano do Ensino Fundamen...
4,Q001,"Até que série seu pai, ou o homem responsável ...",E,"Completou o Ensino Médio, mas não completou a ..."


In [6]:
df.dtypes

ID_QUESTIONARIO    object
PERGUNTA           object
NR_REPOSTA         object
DESC_RESPOSTA      object
dtype: object

In [7]:
tabela = 'QstSocEconomico'

tipo_dado_map = {
    'int64': 'BIGINT',
    'float64': 'FLOAT',
    'object': 'VARCHAR({})',
    'bool': 'BIT',
    'datetime64[ns]': 'DATETIME'
}

def calcular_max_varchar_colunas(df, coluna):
    
    #retornar o comprimento máximo da coluna
    if df[coluna].dtype == 'object':
        return df[coluna].apply(lambda x: len(str(x))).max()
    return None


def gerar_sql_create_table(df, tabela):
    colunas = []
    
    for coluna, tipo in df.dtypes.items():
        if str(tipo) in tipo_dado_map:
            tipo_sql = tipo_dado_map[str(tipo)]
            if 'VARCHAR' in tipo_sql:
                max_length = calcular_max_varchar_colunas(df, coluna)
                if max_length is None:
                    max_length = 255  # Valor padrão caso não seja possível calcular
                tipo_sql = tipo_sql.format(max_length)
            colunas.append(f"[{coluna}] {tipo_sql}")
        else:
            colunas.append(f"[{coluna}] VARCHAR(MAX)")  # Tipo padrão para casos não mapeados
    
    sql_create = f"CREATE TABLE {tabela} (\n" + ",\n".join(colunas) + "\n);"
    return sql_create

# Gerando o script SQL
script_sql_create = gerar_sql_create_table(df, tabela)

# Exibe o script SQL, basta apenas copiar o result e colar no SSMS que será criado a tabela
print(script_sql_create)

CREATE TABLE QstSocEconomico (
[ID_QUESTIONARIO] VARCHAR(4),
[PERGUNTA] VARCHAR(277),
[NR_REPOSTA] VARCHAR(2),
[DESC_RESPOSTA] VARCHAR(437)
);


In [8]:
#Etapa que insere os dados no banco de dados

from tqdm import tqdm

#from sqlalchemy import create_engine
from app.db_config import database_config

# Construa a string de conexão corretamente formatada
connection_string = (
    f"mssql+pyodbc://{database_config['username']}:{database_config['password']}"
    f"@{database_config['host']}/{database_config['database']}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

# Crie o engine do SQLAlchemy
engine = create_engine(connection_string)

chunk_size = 1000  # Defina o tamanho do chunk conforme necessário
total_chunks = (len(df) // chunk_size) + 1  # Calcula o número total de chunks

try:
    with tqdm(total=total_chunks, desc="Inserindo dados no banco") as pbar:
        for start in range(0, len(df), chunk_size):
            end = min(start + chunk_size, len(df))
            chunk = df.iloc[start:end]
            chunk.to_sql('QstSocEconomico', con=engine, if_exists='append', index=False)
            pbar.update(1)  # Atualiza a barra de progresso
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir os dados:", e)

Inserindo dados no banco: 100%|██████████| 1/1 [00:00<00:00, 29.84it/s]

Dados inseridos com sucesso!
